In [38]:
import pandas as pd
import plotly.express as px

# paths = [
#     '/Users/pb/Documents/Github/Tokemak/v2-rebalance-dashboard/mainnet_launch/database/schema/ensure_tables_are_current/using_3rd_party/autoUSD_primary_assets_quotes_slippage_3.5_bps_1_min_sleep.csv',
#     '/Users/pb/Documents/Github/Tokemak/v2-rebalance-dashboard/mainnet_launch/database/schema/ensure_tables_are_current/using_3rd_party/autoUSD_primary_assets_quotes_slippage_3.5_bps.csv',
#     "/Users/pb/Documents/Github/Tokemak/v2-rebalance-dashboard/15_min_auto_usd_combinations2.csv",
#     "/Users/pb/Documents/Github/Tokemak/v2-rebalance-dashboard/15_min_auto_usd_combinations3.csv",
#     "/Users/pb/Documents/Github/Tokemak/v2-rebalance-dashboard/15_min_auto_usd_combinations4.csv",
#     "/Users/pb/Documents/Github/Tokemak/v2-rebalance-dashboard/15_min_auto_usd_combinations.csv"
# ]

# dfs = [pd.read_csv(p) for p in paths]
# df = pd.concat(dfs)

df = pd.read_csv("combined_swap_quotes.csv")
# df.to_csv("combined_swap_quotes.csv", index=True)
excluded = ['frxUSD']
df = df[(~df['sell_symbol'].isin(excluded)) & (~df['buy_symbol'].isin(excluded))].copy()
df['datetime_received'] = pd.to_datetime(df['datetime_received'])
df = df.sort_values('datetime_received')
px.scatter(df, x="datetime_received", y="label", title='Samples over time').show()


df['long_label'] = df['label'] + ' ' + df['sell_amount_norm'].astype(int).astype(str)
df['15_min_chunk'] = df['datetime_received'].dt.floor('15min')

# I think we want 1 for each sample
df[['15_min_chunk', 'long_label']].value_counts().reset_index()['count'].value_counts()
# we only want the samples where there is only one price (eg skip the eariler ones)
df_unique = df.groupby(['15_min_chunk', 'long_label'])[['datetime_received', 'min_buy_amount_price', 'buy_amount_price', 'buy_symbol']].first().reset_index()
               
df_unique[['datetime_received', 'long_label', '15_min_chunk', 'min_buy_amount_price', 'buy_amount_price']]
buy_amount_price_over_time = df_unique.pivot(index='15_min_chunk', columns='long_label', values='buy_amount_price')

In [98]:
def compute_bps_diff_thresholds(buy_amount_price_over_time, column:str, percentile) -> pd.DataFrame:
    one_token_prices = buy_amount_price_over_time[[column]].copy()
    one_token_prices['t-15'] = one_token_prices[column].shift(1)
    one_token_prices['t-30'] = one_token_prices[column].shift(2)
    one_token_prices['t-45'] = one_token_prices[column].shift(3)
    one_token_prices['t-60'] = one_token_prices[column].shift(4)
    one_token_prices['t-120'] = one_token_prices[column].shift(8)
    one_token_prices['t-180'] = one_token_prices[column].shift(12)
    one_token_prices['bps_diff_15_min'] = ((one_token_prices[column] - one_token_prices['t-15']) / one_token_prices['t-15'] * 1_000)
    one_token_prices['bps_diff_30_min'] = ((one_token_prices[column] - one_token_prices['t-30']) / one_token_prices['t-30'] * 1_000)
    one_token_prices['bps_diff_45_min'] = ((one_token_prices[column] - one_token_prices['t-45']) / one_token_prices['t-45'] * 1_000)
    one_token_prices['bps_diff_60_min'] = ((one_token_prices[column] - one_token_prices['t-60']) / one_token_prices['t-60'] * 1_000)
    one_token_prices['bps_diff_120_min'] = ((one_token_prices[column] - one_token_prices['t-120']) / one_token_prices['t-120'] * 1_000)
    one_token_prices['bps_diff_180_min'] = ((one_token_prices[column] - one_token_prices['t-180']) / one_token_prices['t-180'] * 1_000)
    thresholds = one_token_prices.filter(like='bps_diff_').abs().quantile(percentile).rename(column)
    return thresholds

percentile  = .95

def plot_bps_thresholds_by_size():
    for size in df['sell_amount_norm'].unique().astype(int).astype(str):
        all_thresholds = []
        for i, col in enumerate(buy_amount_price_over_time.columns):
            if ' ' + size not in col:
                continue
            thresholds = compute_bps_diff_thresholds(buy_amount_price_over_time, col,percentile  )
            all_thresholds.append(thresholds)
        threshold_df = pd.concat(all_thresholds, axis=1).T
        px.histogram(threshold_df, title=f'{percentile}th percentile of bps diff for size ' + size).show()


def plot_bps_diffs_by_label():
    for label in df_unique['buy_symbol'].unique():
        all_thresholds = []

        for i, col in enumerate(buy_amount_price_over_time.columns):
            if str(label) in col:
                thresholds = compute_bps_diff_thresholds(buy_amount_price_over_time, col, percentile)
                all_thresholds.append(thresholds)
            
        if len(all_thresholds) == 0:
            print('No data for label', label)
            continue

        threshold_df = pd.concat(all_thresholds, axis=1).T
        px.ecdf(threshold_df, title=f'{percentile}th percentile of bps diff for label ' + label, range_x=[0,1]).show()

def plot_all_bps_diffs():
    all_thresholds = []
    for i, col in enumerate(buy_amount_price_over_time.columns):
        thresholds = compute_bps_diff_thresholds(buy_amount_price_over_time, col, percentile)
        all_thresholds.append(thresholds)
    threshold_df = pd.concat(all_thresholds, axis=1).T
    px.ecdf(threshold_df, title=f'{percentile}th percentile of bps diff for all').show()

plot_all_bps_diffs()
plot_bps_diffs_by_label()


In [ ]:
# to

# get spot price in usd for each token at that time
# use the same block when you can, pick a block after it 

#actaully you 

# datetime -> block, 
#get spot price of each token in usd at that block


In [99]:
# plot_bps_thresholds_by_size()

In [ ]:
# I'm doing this wrong, I need to get the swap cost in bps, not the price change in bps
# 

In [72]:
df_unique['buy_symbol'].unique()

array(['GHO', 'USDC', 'USDT', 'USDe', 'crvUSD', 'FRAX'], dtype=object)

In [73]:
.0001 * 100_000

10.0

In [74]:
[1- (1 * .00002), 1 + (1 * .00002)]

[0.99998, 1.00002]

In [75]:
# to get, 

# everything, 

# what are the outliers



# On ~ September 23 - 24 ish

- 90% of prices at Time T=0 (execpt for 2 outliers) are with .2 bps of the price at T-15 for the permutations of ['USDT', 'FRAX', 'GHO', 'USDe', 'USDC', 'crvUSD']
- slippage bps setting in tokemak swap API when set to a high value gives a very small amount of bps of slippage. eg if we set slippage is 50% then min buy amount is ~50% of the price of the buy amount price



In [76]:
100_000 - (100_000 *.00003), 100_000 + (100_000 *.00003)

(99997.0, 100003.0)